# Volatility Scaling & Portfolio Analysis

This notebook demonstrates how to:
1. Load and validate data.
2. Handle missing data (short vs. long gaps).
3. Adjust returns to a target volatility in-sample, then apply the same scaling out-of-sample.
4. Compute Sharpe, Sortino, Max Drawdown.
5. Provide multiple fund selection modes (all, random sample, manual).
6. Calculate portfolio results (equal-weight and custom-weight).
7. Output in-sample and out-of-sample results to Excel with formatting.

**Note**: The manual fund selection and custom weights features are partially implemented. In a real interactive workflow, you would wire widget selections and weights into the final analysis.

In [1]:
# ============ 1 · SETUP ============

import sys
import logging
import warnings
from io import BytesIO

import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser
import xlsxwriter           # required by export_to_excel()

# ── Logging ──────────────────────────────────────────────────────
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format="%(levelname)s: %(message)s"
)
logging.info("Volatility-Scaling & Portfolio-Analysis notebook initialised.")

# ── Warning filters ─────────────────────────────────────────────
# Silence pandas’ date-parsing fallback notice.
warnings.filterwarnings(
    "ignore",
    message=r"Could not infer format.*falling back to `dateutil`",
    category=UserWarning,
    module="pandas"
)

print("Setup complete.")

INFO: Volatility-Scaling & Portfolio-Analysis notebook initialised.
Setup complete.


## 2. Data Loading
Here we create options to load a dataset from a local file or a GitHub repository.

In [2]:
# ============ 2 · DATA LOADING ============

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser
import logging

# ── helpers ────────────────────────────────────────────────────────
def load_csv(path_or_url: str) -> pd.DataFrame:
    """Read a CSV with pandas. Let pandas raise if it fails."""
    logging.info("Reading CSV: %s", path_or_url)
    df = pd.read_csv(path_or_url, engine="python")
    if "Date" not in df.columns:
        raise ValueError("No 'Date' column found.")
    return df

def identify_risk_free_fund(df: pd.DataFrame) -> str:
    """Return column with the smallest stdev (ex-Date)."""
    target_cols = df.columns.drop("Date")
    rf_col = (df[target_cols].std().idxmin())
    logging.info("Risk-free column = %s", rf_col)
    return rf_col

# ── minimal UI ─────────────────────────────────────────────────────
src_toggle  = widgets.ToggleButtons(
    options=[("Local file", "local"), ("GitHub raw URL", "url")],
    description="Source:"
)
file_picker = FileChooser(title="Pick .csv")
url_box     = widgets.Text(description="Raw-URL:", placeholder="https://…/file.csv")
load_btn    = widgets.Button(description="Load", button_style="success")
status_out  = widgets.Output()

def _toggle_src(change):
    mode = change["new"]
    file_picker.layout.display = "block" if mode == "local" else "none"
    url_box.layout.display     = "block" if mode == "url"   else "none"
src_toggle.observe(_toggle_src, names="value")
_toggle_src({"new": src_toggle.value})        # initialise visibilities

session = {"df": None, "rf_col": None}        # lightweight global

def _load_clicked(_):
    with status_out:
        clear_output()
        try:
            if src_toggle.value == "local":
                if not file_picker.selected:
                    print("⚠️ choose a file first"); return
                path = file_picker.selected
            else:
                path = url_box.value.strip()
                if not path.lower().endswith(".csv"):
                    print("⚠️ URL must end with .csv"); return
            df = load_csv(path)
            rf = identify_risk_free_fund(df)
            session["df"], session["rf_col"] = df, rf
            print(f"✅ Loaded {len(df):,} rows × {df.shape[1]} cols")
        except Exception as e:
            session["df"] = None
            print("❌ Failed:", e)


## 3. Utility Functions
Here we define date parsing, consecutive gap checks, data filling, risk-free identification, return calculations, etc.

In [3]:
# ============ 3 · UTILITY FUNCTIONS ============

import numpy as np
import pandas as pd
import logging
from typing import List, Dict, Optional

M_PER_YEAR = 12   # monthly data

# ── Basic statistics ────────────────────────────────────────────────────
def annualize_return(series: pd.Series) -> float:
    s = series.dropna()
    if s.empty: return np.nan
    geom = (1 + s).prod() ** (M_PER_YEAR / len(s)) - 1
    return geom

def annualize_volatility(series: pd.Series) -> float:
    s = series.dropna()
    return s.std(ddof=0) * np.sqrt(M_PER_YEAR) if not s.empty else np.nan

def sharpe_ratio(series: pd.Series, rf_series: pd.Series) -> float:
    excess = (series - rf_series).dropna()
    vol = annualize_volatility(excess)
    return annualize_return(excess) / vol if vol else np.nan

def sortino_ratio(series: pd.Series, rf_series: pd.Series) -> float:
    excess = (series - rf_series).dropna()
    neg = excess[excess < 0]
    if neg.empty: return np.nan
    downside_dev = neg.std(ddof=0) * np.sqrt(M_PER_YEAR)
    return annualize_return(excess) / downside_dev if downside_dev else np.nan

def max_drawdown(series: pd.Series) -> float:
    s = (1 + series).dropna().cumprod()
    peak = s.cummax()
    draw = (s / peak) - 1
    return draw.min() if not draw.empty else np.nan

# ── Portfolio helpers ──────────────────────────────────────────────────
def calc_portfolio_returns(weights: np.ndarray,
                           df_returns: pd.DataFrame) -> pd.Series:
    """Matrix dot product w/ alignment; assumes df already sorted & NaNs dropped."""
    return df_returns.dot(weights)

# ── Fund-selection helper ──────────────────────────────────────────────
def select_funds(df: pd.DataFrame,
                 rf_col: str,
                 fund_columns: List[str],
                 in_sdate: pd.Timestamp,
                 in_edate: pd.Timestamp,
                 out_sdate: pd.Timestamp,
                 out_edate: pd.Timestamp,
                 selection_mode: str = "all",
                 random_n: Optional[int] = None,
                 seed: Optional[int] = None) -> List[str]:
    """
    production-grade fund selector.
      * full-history rule: non-NA for entire in + out windows
      * no >3 consecutive NA rule across full history
    """
    rng = np.random.default_rng(seed)
    full_hist = []

    mask_full = (df["Date"] >= in_sdate) & (df["Date"] <= out_edate)
    slice_df  = df.loc[mask_full, fund_columns]

    for col in slice_df.columns:
        col_ser = slice_df[col]
        if col_ser.isna().any():
            # reject if >3 consecutive NAs
            if (col_ser.isna().astype(int).groupby(col_ser.notna().cumsum())
                .sum().max() > 3):
                continue
        full_hist.append(col)

    if not full_hist:
        return []

    if selection_mode == "all":
        return full_hist
    if selection_mode == "random":
        if random_n is None or random_n >= len(full_hist):
            return full_hist
        return list(rng.choice(full_hist, size=random_n, replace=False))
    raise ValueError("selection_mode must be 'all', 'random', or handled by caller.")

# ── compute_stats wrapper for reuse ────────────────────────────────────
def compute_stats(series: pd.Series, rf_series: pd.Series) -> tuple:
    """Return (R, V, Sharpe, Sortino, MDD)"""
    return (
        annualize_return(series),
        annualize_volatility(series),
        sharpe_ratio(series, rf_series),
        sortino_ratio(series, rf_series),
        max_drawdown(series)
    )

logging.info("Utility-function suite loaded.")


INFO: Utility-function suite loaded.


#### 4. Fund Selection
Filters out columns that represent the risk-free rate or contain "index" in the name, then handles the selection mode (all, random, or manual).

In [4]:
# ============ 4 · FUND-SELECTION UI ============

import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import logging

# ---------------- UI widgets -------------------
mode_dd = widgets.Dropdown(
    options=[("All funds", "all"),
             ("Random sample", "random"),
             ("Manual pick",   "manual")],
    value="all", description="Mode:"
)
rand_n  = widgets.BoundedIntText(
    5, min=2, max=50, step=1, description="Sample N:"
)
fund_table = widgets.VBox([])        # will hold checkboxes + weights
total_lbl  = widgets.Label("Total = 0 %")

run_btn    = widgets.Button(
    description="Confirm selection", button_style="success"
)
ui_out     = widgets.Output()

# ---------- visibility helpers -----------------
def _toggle_mode(change=None):
    m = mode_dd.value
    rand_n.layout.display  = "block" if m == "random" else "none"
    fund_table.layout.display = "block" if m == "manual" else "none"
    total_lbl.layout.display  = "block" if m == "manual" else "none"
mode_dd.observe(_toggle_mode, names="value")
_toggle_mode()

# ---------- build manual-fund table ------------
def _build_table(*_):
    df = session.get("df")
    rf = session.get("rf_col")
    if df is None:
        with ui_out:
            clear_output()
            print("⚠️  Load data first.")
        return

    # candidate list = every non-index, non-Date, non-rf column
    cand = [c for c in df.columns if c not in ["Date", rf]]
    valid = select_funds(
        df, rf, cand,
        df["Date"].min(), df["Date"].max(),
        df["Date"].min(), df["Date"].max(),
        "all"
    )
    if not valid:
        with ui_out: clear_output(); print("❌ No eligible funds."); return

    rows, cb_list, wt_list = [], [], []

    def _update_total(*_):
        tot = sum(wt.value for cb, wt in zip(cb_list, wt_list) if cb.value)
        total_lbl.value = f"Total = {tot} %"

    for f in valid:
        cb = widgets.Checkbox(description=f, layout=widgets.Layout(width="200px"))
        wt = widgets.BoundedIntText(
            0, min=0, max=100, step=1,
            layout=widgets.Layout(width="60px"), disabled=True
        )
        def _toggle(ch, box=wt):
            box.disabled = not ch["new"]
            if box.disabled: box.value = 0
            _update_total()
        cb.observe(_toggle, names="value")
        wt.observe(_update_total, names="value")

        cb_list.append(cb); wt_list.append(wt)
        rows.append(widgets.HBox([cb, wt]))

    fund_table.children = rows
    _update_total()

# ---------- confirm-selection handler ----------
def _confirm(_):
    with ui_out:
        clear_output()

        df = session.get("df"); rf = session.get("rf_col")
        if df is None:
            print("⚠️  Load data first."); return

        mode = mode_dd.value
        if mode == "all":
            session["selected_funds"] = [c for c in df.columns if c not in ["Date", rf]]
            session["custom_weights"] = None
            print(f"✔ Selected ALL funds ({len(session['selected_funds'])})")

        elif mode == "random":
            n = rand_n.value
            pool = [c for c in df.columns if c not in ["Date", rf]]
            if n > len(pool):
                print("⚠️  Sample N exceeds pool size."); return
            session["selected_funds"] = list(np.random.choice(pool, size=n, replace=False))
            session["custom_weights"] = None
            print(f"✔ Random pick: {session['selected_funds']}")

        else:  # manual
            sel, wts = [], {}
            for row in fund_table.children:
                cb, wt = row.children
                if cb.value:
                    sel.append(cb.description)
                    wts[cb.description] = wt.value
            if not sel:
                print("⚠️  Tick at least one fund."); return
            if sum(wts.values()) != 100:
                print("⚠️  Weights must sum to 100 %."); return
            session["selected_funds"] = sel
            session["custom_weights"] = wts
            print(f"✔ Manual pick: {sel}")

run_btn.on_click(_confirm)

# auto-build table once when user first switches to Manual
mode_dd.observe(lambda ch: _build_table() if ch["new"] == "manual" else None,
                names="value")


## 5. Custom Weights
Displays an integer text widget for each fund, requiring the sum of weights to be 100.

In [5]:
# ============ 5 · CUSTOM WEIGHTS HELPERS ============

import numpy as np
import logging
from typing import Dict, List, Tuple, Optional

def validate_weights(weights: Dict[str, int | float],
                     fund_list: List[str]) -> None:
    """
    Ensure keys match fund_list and values sum to 100 (percent integers).
    Raises ValueError on any issue.
    """
    if set(weights) != set(fund_list):
        raise ValueError("Weight keys don’t match selected funds.")
    total = sum(weights.values())
    if total != 100:
        raise ValueError(f"Weights sum to {total} instead of 100 %.")

def prepare_weights(selected_funds: List[str],
                    custom_weights: Optional[Dict[str, int]]) -> Tuple[Dict[str, float], np.ndarray]:
    """
    Convert user dictionary of integer % to:
      • dict of decimals
      • NumPy vector aligned to selected_funds
    If custom_weights is None, returns equal weights.
    """

    n = len(selected_funds)
    if custom_weights is None:
        equal = 1 / n
        dec_dict = {f: equal for f in selected_funds}
        vec = np.full(n, equal)
        logging.info("Equal weights applied (%d funds).", n)
        return dec_dict, vec

    # manual mode
    validate_weights(custom_weights, selected_funds)
    dec_dict = {f: pct / 100.0 for f, pct in custom_weights.items()}
    vec = np.array([dec_dict[f] for f in selected_funds])
    logging.info("Custom weights accepted.")
    return dec_dict, vec


## 6. Analysis (In-Sample & Out-of-Sample)
The `run_analysis` function orchestrates the entire process:
- Validates date inputs.
- Converts 'Date' column.
- Identifies risk-free column.
- Fills short gaps.
- Selects funds.
- Computes in-sample scaling factors and applies them in- and out-of-sample.
- Computes individual fund stats and portfolio stats.

## 7. Excel Export
Creates an Excel file with two sheets (In-Sample, Out-of-Sample) and two tables per sheet (Equal-weight and User-weight).

In [6]:
# ============ 7 · EXPORT TO EXCEL (weight-format fix) ============

import xlsxwriter, numpy as np, pandas as pd
from io import BytesIO
import logging

def export_to_excel(results: dict,
                    full_df: pd.DataFrame,
                    fname: str,
                    in_start: str, in_end: str,
                    out_start: str, out_end: str) -> None:
    """
    Workbook writer.
    • Funds in indices_list are skipped.
    • Weight % shows:
        – whole integer if pct ≥ 1
        – two decimals if pct < 1 (e.g., 0.98 %)
    """

    # ---------- helpers --------------------------------------------------
    def safe(v):          # NaN/inf → blank
        return "" if (pd.isna(v) or not np.isfinite(v)) else v

    def ok(t1, t2):       # at least one finite metric
        return any(np.isfinite(x) for x in t1 + t2)

    def pctify(t):
        r, v, s, so, m = t
        return [r*100, v*100, s, so, m*100]

    indices_set = set(results.get("indices_list", []))

    # ---------- workbook -------------------------------------------------
    buf = BytesIO()
    wb  = xlsxwriter.Workbook(buf, {"in_memory": True})
    ws  = wb.add_worksheet("Summary")

    bold   = wb.add_format({"bold": True})
    int0   = wb.add_format({"num_format": "0"})
    num2   = wb.add_format({"num_format": "0.00"})
    red    = wb.add_format({"num_format": "0.00", "font_color": "red"})

    # ---------- header ---------------------------------------------------
    ws.write(0, 0, "Vol-Adj Trend Analysis", bold)
    ws.write(1, 0, f"In-Sample:  {in_start} → {in_end}")
    ws.write(2, 0, f"Out-Sample: {out_start} → {out_end}")

    row = 4
    ws.write(row, 0, "Portfolio returns", bold); row += 1

    cols = ["Name", "Weight %",
            "R (IN) %", "V (IN) %", "Sharpe (IN)", "Sortino (IN)", "MDD (IN) %",
            "R (OUT) %", "V (OUT) %", "Sharpe (OUT)", "Sortino (OUT)", "MDD (OUT) %"]
    ws.write_row(row, 0, cols, bold); row += 1

    # ---------- inner writer --------------------------------------------
    def write_row(r, name, wt_dec_or_blank, tin, tout, emph=False):
        ws.write(r, 0, name, bold if emph else None)

        # Weight % column
        if wt_dec_or_blank == "":
            ws.write(r, 1, "")
        else:
            pct = wt_dec_or_blank*100 if wt_dec_or_blank <= 1 else wt_dec_or_blank
            fmt = int0 if pct >= 1 else num2
            ws.write(r, 1, pct, fmt)

        vals = pctify(tin) + pctify(tout)
        fmts = [num2, num2, num2, num2, red,
                num2, num2, num2, num2, red]
        for c, (v, f) in enumerate(zip(vals, fmts), start=2):
            ws.write(r, c, safe(v), f)

    # ---------- weights dicts -------------------------------------------
    ew_w = results.get("ew_weights", {})
    uw_w = results.get("fund_weights", {}) or ew_w

    # ---------- portfolio rows ------------------------------------------
    write_row(row, "Equal-Weight", 1,
              results["in_ew_stats"], results["out_ew_stats"], True); row += 1
    write_row(row, "User-Weight",  1,
              results["in_user_stats"], results["out_user_stats"], True); row += 2

    # ---------- fund rows (eligible only) -------------------------------
    ws.write(row, 0, "Funds", bold); row += 1
    kept = 0
    for f in results["selected_funds"]:
        if f in indices_set:
            continue
        tin  = results["in_sample_stats"][f]
        tout = results["out_sample_stats"][f]
        if not ok(tin, tout):
            continue
        write_row(row, f, uw_w.get(f, 0), tin, tout)
        row += 1
        kept += 1

    # ---------- index rows ----------------------------------------------
    if results.get("index_stats"):
        ws.write(row, 0, "INDEX", bold); row += 1
        for idx, sd in results["index_stats"].items():
            write_row(row, idx, "", sd["in_sample"], sd["out_sample"], True)
            row += 1

    # ---------- raw data tab --------------------------------------------
    ws2 = wb.add_worksheet("Raw Data")
    ws2.write_row(0, 0, full_df.columns.tolist(), bold)
    for r, (_, ser) in enumerate(full_df.iterrows(), start=1):
        ws2.write_row(
            r, 0,
            [d.strftime("%Y-%m-%d") if isinstance(d, pd.Timestamp)
             else "" if (isinstance(d, (float, int)) and (pd.isna(d) or not np.isfinite(d)))
             else d
             for d in ser]
        )

    wb.close()
    with open(fname, "wb") as f:
        f.write(buf.getvalue())

    logging.info("Workbook saved → %s (funds written: %d)", fname, kept)


## 8. Run Parameters,Widgets & User Inputs
Here we define some IPython widgets for in-sample/out-of-sample dates, target volatility, monthly cost, etc.

### Using This Notebook
1. Run all cells.
2. Call `demo_run()` in a new cell to see a quick example with dummy data.
3. To use your own data, load it into a DataFrame (make sure it has a 'Date' column and decimal returns in other columns), then call `run_analysis()` and `export_to_excel()`.
4. For interactive selection, do:
   ```python
   display(ui_inputs)
   ```
   Then wire the `apply_button` to a callback function that reads the widget values and runs `run_analysis()`.
5. For custom weights, call:
   ```python
   my_weights = get_custom_weights(selected_funds)
   ```
   Then pass `my_weights` into your logic.


In [10]:
# ===============================================================
#              ONE-STOP ANALYSIS UI  (eligibility-synced)
# ===============================================================

import pandas as pd, numpy as np, ipywidgets as widgets, logging
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser

# ───────── session dict ─────────
session = {"df": None, "rf_col": None,
           "selected_funds": None, "custom_weights": None}

# ───────── 1 · DATA SOURCE ───────
src_toggle = widgets.ToggleButtons(
    options=[("Local file", "local"), ("GitHub raw URL", "url")],
    description="Source:"
)
file_chooser = FileChooser(title="Pick .csv")
url_box  = widgets.Text(description="Raw-URL:", placeholder="https://…/file.csv")
load_btn = widgets.Button(description="Load CSV", button_style="success")
load_out = widgets.Output()

def _show_src(ch):
    file_chooser.layout.display = "block" if ch["new"] == "local" else "none"
    url_box.layout.display      = "block" if ch["new"] == "url"   else "none"
src_toggle.observe(_show_src, names="value")
_show_src({"new": src_toggle.value})        # initialise visibility

def _load(_):
    with load_out:
        clear_output()
        try:
            path = file_chooser.selected if src_toggle.value == "local" else url_box.value.strip()
            if not path:
                print("⚠️ choose file / URL"); return
            if src_toggle.value == "url" and not path.lower().endswith(".csv"):
                print("⚠️ URL must end with .csv"); return
            df = load_csv(path)
            rf = identify_risk_free_fund(df)
            session.update(df=df, rf_col=rf,
                           selected_funds=None, custom_weights=None)
            print(f"✅ Loaded {len(df):,} rows × {df.shape[1]} cols  |  RF → {rf}")
        except Exception as e:
            session["df"] = None
            print("❌", e)
load_btn.on_click(_load)

# ───────── 2 · PARAMETERS ─────────
index_cnt   = widgets.BoundedIntText(0, min=0, max=10, description="# Indices:")
in_start    = widgets.Text("2005-07", description="In Start:")
in_end      = widgets.Text("2008-06", description="In End:")
out_start   = widgets.Text("2008-07", description="Out Start:")
out_end     = widgets.Text("2009-06", description="Out End:")
target_vol   = widgets.FloatText(0.25,  description="Target Vol:")
monthly_cost = widgets.FloatText(0.0033, description="Monthly Cost:")

# ───────── 3 · FUND SELECTION ─────
mode_dd = widgets.Dropdown(
    options=[("All funds", "all"),
             ("Random sample", "random"),
             ("Manual pick",   "manual")],
    value="all", description="Mode:"
)
rand_n      = widgets.BoundedIntText(5, min=2, max=100, description="Sample N:")
fund_table  = widgets.VBox([])
total_lbl   = widgets.Label("Total = 0 %")

def _toggle_vis(_=None):
    rand_n.layout.display         = "block" if mode_dd.value == "random" else "none"
    show                           = "block" if mode_dd.value == "manual" else "none"
    fund_table.layout.display      = show
    total_lbl.layout.display       = show
mode_dd.observe(_toggle_vis, names="value")
_toggle_vis()

def _eligible_pool() -> list[str]:
    df, rf = session["df"], session["rf_col"]
    if df is None: return []
    cand = [c for c in df.columns if c not in ["Date", rf]]
    return select_funds(
        df, rf, cand,
        in_start.value + "-01", in_end.value + "-01",
        out_start.value + "-01", out_end.value + "-01",
        "all"
    )

def _build_manual(*_):
    if mode_dd.value != "manual" or session["df"] is None: return
    valid = _eligible_pool()
    rows, cbxs, wbx = [], [], []
    def _update_total(*_):
        total_lbl.value = f"Total = {sum(w.value for c,w in zip(cbxs,wbx) if c.value)} %"
    fund_table.children = []           # clear previous
    for f in valid:
        cb = widgets.Checkbox(description=f, layout=widgets.Layout(width="200px"))
        wt = widgets.BoundedIntText(0, min=0, max=100,
                                    layout=widgets.Layout(width="60px"), disabled=True)
        cb.observe(lambda ch, box=wt: (setattr(box,"disabled",not ch["new"]), _update_total()),
                   names="value")
        wt.observe(_update_total, names="value")
        cbxs.append(cb); wbx.append(wt); rows.append(widgets.HBox([cb, wt]))
    fund_table.children = rows
    _update_total()

# rebuild manual table on mode change and date edits
mode_dd.observe(lambda ch: _build_manual() if ch["new"]=="manual" else None,
                names="value")
for w in (in_start, in_end, out_start, out_end):
    w.observe(_build_manual, names="value")

# ───────── 4 · RUN ANALYSIS ───────
run_btn = widgets.Button(description="Run Analysis", button_style="success")
run_out = widgets.Output(layout={"border":"1px solid #999",
                                 "height":"340px", "overflow_y":"auto"})

def _run(_):
    with run_out:
        clear_output()
        df, rf = session["df"], session["rf_col"]
        if df is None:
            print("⚠️ Load data first"); return

        # parse dates
        try:
            _ = [pd.to_datetime(s+"-01", format="%Y-%m-%d", errors="raise")
                 for s in (in_start.value, in_end.value, out_start.value, out_end.value)]
        except Exception:
            print("❌ dates must be YYYY-MM"); return

        # indices list
        idx_n = index_cnt.value
        indices_list = df.columns.drop("Date").to_list()[-idx_n:] if idx_n else []

        valid = _eligible_pool()
        if not valid:
            print("❌ No funds have full data in both windows."); return

        # selection
        if mode_dd.value == "all":
            sel, custom_w = valid, None
        elif mode_dd.value == "random":
            if rand_n.value > len(valid):
                print(f"❌ Only {len(valid)} eligible funds; Sample N exceeds that."); return
            sel, custom_w = list(np.random.choice(valid, rand_n.value, replace=False)), None
        else:
            sel, custom_w = [], {}
            if not fund_table.children: _build_manual()
            for row in fund_table.children:
                cb, wt = row.children
                if cb.value:
                    sel.append(cb.description)
                    custom_w[cb.description] = wt.value
            if not sel:
                print("⚠️ Tick at least one fund."); return
            if sum(custom_w.values()) != 100:
                print("⚠️ Weights must sum to 100 %."); return

        session.update(selected_funds=sel, custom_weights=custom_w)

        # weights
        try:
            w_dict, w_vec = prepare_weights(sel, custom_w)
        except ValueError as e:
            print("❌", e); return

        # analysis
        res = run_analysis(
            df, sel, w_vec, w_dict, rf,
            in_start.value, in_end.value,
            out_start.value, out_end.value,
            target_vol.value, monthly_cost.value,
            indices_list
        )

        # export
        print("✅ analysis complete |", len(sel), "funds")
        fname = f"IS_{in_start.value}_{out_start.value}.xlsx"
        export_to_excel(res, df, fname,
                        in_start.value, in_end.value,
                        out_start.value, out_end.value)
        print("Workbook saved as", fname)

run_btn.on_click(_run)

# ───────── DISPLAY PANEL ─────────
display(widgets.VBox([
    widgets.HTML("<h4>1. Load data</h4>"),
    src_toggle, file_chooser, url_box, load_btn, load_out,
    widgets.HTML("<hr><h4>2. Parameters</h4>"),
    widgets.HBox([index_cnt]),
    widgets.HBox([in_start, in_end, out_start, out_end]),
    widgets.HBox([target_vol, monthly_cost]),
    widgets.HTML("<hr><h4>3. Fund selection</h4>"),
    widgets.HBox([mode_dd, rand_n]),
    fund_table, total_lbl,
    widgets.HTML("<hr>"),
    run_btn,
    run_out
]))
